In [15]:
import pandas as pd

viv_x_mzas = pd.read_csv("/tmp/pilar_vivs_x_mnza.csv")
print(viv_x_mzas)

viv_x_lado = pd.read_csv("/tmp/pilar_vivs_x_lado.csv")
print(viv_x_lado)

mzas_ady = pd.read_csv("/tmp/pilar_mzas_adyacentes.csv", dtype={'mza': object, 'mza_ady': object})
print(mzas_ady)

lados_ady = pd.read_csv("/tmp/pilar_lados_adyacentes.csv"
                        , dtype={'mza': object, 'd_lado': object, 'v_mza': object, 'v_lado': object, 'c_mza': object, 'c_lado': object})
print(lados_ady)






      depto  frac  radio manzana  viviendas
0       638     1      1     001         10
1       638     1      1     003         20
2       638     1      1     004          6
3       638     1      1     005          1
4       638     1      1     006          2
5       638     1      1     007          2
6       638     1      1     008          5
7       638     1      1     009          7
8       638     1      1      01          2
9       638     1      1     010          8
10      638     1      1     011          5
11      638     1      1     012          2
12      638     1      1     013          5
13      638     1      1      03          2
14      638     1      2       0          8
15      638     1      2     002          1
16      638     1      2     003         25
17      638     1      2     004         12
18      638     1      2     005         10
19      638     1      2     007         12
20      638     1      2     008         16
21      638     1      2     009

In [1]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(database = "halpe", user = "halpe", password = "halpe", host = "172.26.67.239", port = "5432")
print "Opened database successfully"

cur = conn.cursor()
sql = ("select frac, radio, ccalle::integer, ncalle, hnpuerta, hpiso, mza::integer, lado, seg, count(*) "
    "from prueba_piloto.pilar "
    "where (frac, radio, mza) in ( "
        "select frac, radio, mza "
        "from prueba_piloto.pilar "
        "where prueba_piloto.isnumeric(mza) and mza::integer != 0 "
        "group by frac, radio, mza "
        "having count(*) between 60 and 40 "
        "or count(*) > 90) "
    "group by frac, radio, ccalle::integer, ncalle, hnpuerta, hpiso, mza::integer, lado, seg "
    "order by frac, radio, mza::integer, seg, ccalle, hnpuerta"
    )
cur.execute(sql)
colnames = [desc[0] for desc in cur.description]
df = pd.DataFrame(cur.fetchall(), columns = colnames)


print(df)

print "Operation done successfully";
conn.close()

Opened database successfully
     frac radio  ccalle            ncalle hnpuerta hpiso  mza  lado seg  count
0      01    11       0            DURANO    01102     0    1  None  01      1
1      01    11       0           FIORINI    01719     0    1  None  01      1
2      01    11       0           FIORINI    01731     0    1  None  01      1
3      01    11       0           FIORINI    01743     0    1  None  01      1
4      01    11       0            DURAND    01744     0    1  None  01      1
5      01    11       0            DURAND    01756     0    1  None  01      2
6      01    11       0            DURAND    01782     0    1  None  01      2
7      01    11       0             GAITO    01794     0    1  None  01      1
8      01    11       0             GAITO    02226     0    1  None  01      1
9      01    11       0             GAITO    02236     0    1  None  01      1
10     01    11       0          CALCAGNO    02237     0    1  None  01      1
11     01    11       0

In [211]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(database = "halpe", user = "halpe", password = "halpe", host = "172.26.67.239", port = "5432")
print "Opened database successfully"
cur = conn.cursor()

sql = ("select mza::integer, v_mza::integer "
        "from segmenta.pilar_mzas_adjacentes "
        "where frac='07' and radio='01'"  #vamos a trabajar con este ejemplo
        "order by mza, v_mza;"
      )
cur.execute(sql)
colnames = [desc[0] for desc in cur.description]
adyacencias_x_manzanas = pd.DataFrame(cur.fetchall(), columns = colnames)

print(adyacencias_x_manzanas)

sql = ("select mza, sum(viviendas) as viviendas "
       "from segmenta.pilar_vivs_x_lado "
       "where (frac, radio, mza) in (select frac, radio, mza::integer from segmenta.pilar_mzas_adjacentes) "
       "and frac='07' and radio='01'"
       "group by mza "
       "order by mza;"
      )
cur.execute(sql)
colnames = [desc[0] for desc in cur.description]
viviendas_x_manzanas = pd.DataFrame(cur.fetchall(), columns = colnames)

print(viviendas_x_manzanas)


print "Operation done successfully";
conn.close()

Opened database successfully
     mza  v_mza
0      1      2
1      2      1
2      2      3
3      2      5
4      3      2
5      3      4
6      3      8
7      4      3
8      4      5
9      4      7
10     5      2
11     5      4
12     5      6
13     5      7
14     5     11
15     5     12
16     6      5
17     6      7
18     6     11
19     7      4
20     7      5
21     7      6
22     7      8
23     7     10
24     8      3
25     8      7
26     8      9
27     9      8
28     9     10
29     9     15
..   ...    ...
84    22     21
85    22     23
86    22     26
87    22     27
88    23     17
89    23     22
90    23     24
91    23     25
92    24     17
93    24     23
94    25     23
95    25     26
96    25    970
97    26     22
98    26     25
99    26     27
100   26    970
101   27     22
102   27     26
103   27     28
104   27    970
105   28     21
106   28     27
107   28    970
108  970     12
109  970     21
110  970     25
111  970     26
112  970   

In [205]:
# algortimo a manzanas

cantidad_de_viviendas_deseada = 36

def penalidad_de_bloque(bloque):
    return (sum(bloque) - cantidad_de_viviendas_deseada)**2
def costo_segmentacion(segmentacion):
    return sum(map(penalidad_de_bloque, segmentacion))


print(penalidad_de_bloque([36])) 
print(penalidad_de_bloque([0]))
print(penalidad_de_bloque([0, 0,]))
print(penalidad_de_bloque([0, 0, 36]))

print(costo_segmentacion([[36], [36]]))
print(costo_segmentacion([[36], [0]]))
print(costo_segmentacion([[35], [37]]))
print(costo_segmentacion([[34], [38]]))


0
1296
1296
0
0
1296
2
8


In [229]:
mzas = pd.merge(viviendas_x_manzanas, adyacencias_x_manzanas, on= ['mza'], how = 'right', sort = True).fillna(0)
mzas_gr = mzas.groupby(['mza', 'viviendas'])['v_mza'].apply(list).reset_index()

manzanas = mzas_gr.set_index(['mza']).to_dict()

print(manzanas['v_mza'])
for mza, adyacentes in enumerate(manzanas['v_mza']):
    print(adyacentes)
for mza, viviendas in enumerate(manzanas['viviendas']):
    print mza, viviendas
    
    



{1: [2], 2: [1, 3, 5], 3: [2, 4, 8], 4: [3, 5, 7], 5: [2, 4, 6, 7, 11, 12], 6: [5, 7, 11], 7: [4, 5, 6, 8, 10], 8: [3, 7, 9], 9: [8, 10, 15], 10: [7, 9, 11, 14], 11: [5, 6, 10, 12, 13], 12: [5, 11, 13, 20, 21, 970], 13: [11, 12, 14, 19, 20], 14: [10, 13, 15, 18], 15: [9, 14, 16, 17], 16: [15, 17], 17: [15, 16, 18, 23, 24], 18: [14, 17, 19, 22], 19: [13, 18, 20, 21], 20: [12, 13, 19, 21], 21: [12, 19, 20, 22, 28, 970], 22: [18, 21, 23, 26, 27], 23: [17, 22, 24, 25], 24: [17, 23], 25: [23, 26, 970], 26: [22, 25, 27, 970], 27: [22, 26, 28, 970], 28: [21, 27, 970], 970: [12, 21, 25, 26, 27, 28]}


TypeError: 'numpy.int64' object is not iterable

In [209]:
viv_list = mzas_gr.groupby(['frac', 'radio'])['viviendas'].apply(list)
mzas_list = mzas_gr.groupby(['frac', 'radio'])['mza'].apply(list)

print(mzas_list['07']['01'])

radio = viv_list['07']['01']
#print(radio)

una_manzana_un_bloque = []
for mza in radio:
    una_manzana_un_bloque.append([mza])
#print(una_manzana_un_bloque)
print(costo_segmentacion(una_manzana_un_bloque))
print(costo_segmentacion([radio]))


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 970]
25043
137641
